In [3]:

import numpy as np
import pandas as pd 
import random
import os
import pickle
from tqdm import tqdm
import multiprocessing

In [4]:
def reservoir_sampling(iterable, n, seed=1234):
    np.random.seed(seed)
    i = 0
    pool = []
    for item in iterable:
        if len(pool) < n:
            pool.append(item)
        else:
            k = np.random.randint(0, i)
            if k < n:
                pool[k] = item

        i += 1
    return pool

In [5]:
class ReservoirSampler:
    def __init__(self, k, rng_seed):
        self.k = k
        self.total = 0
        self.values = []
        self.rng = random.Random(rng_seed)

    def add(self, value):
        if len(self.values) < self.k:
            self.values.append(value)
        else:
            r = self.rng.randint(0, self.total)
            if r < self.k:
                self.values[r] = value

        self.total += 1

In [1]:
path = '/local-scratch/nigam/projects/jfries/crfm/datasets/pretraining/shc/markup_codes_notes_desc_dedup_x8_v1/'
import os
files = os.listdir(path)

In [5]:
train_file='valid.19.1684831482.jsonl.gz'
import pandas as pd
df = pd.read_json(os.path.join(path, train_file), lines=True, compression='gzip')

In [6]:
df

,uid,person_id,split,text
0,3485184,80499692,valid,"<record><visit type=""NULL"" start=""10/18/2014 0..."
1,3485199,76742660,valid,"<record><visit type=""NULL"" start=""08/21/2018 1..."
2,3485210,80554491,valid,"<record><visit type=""NULL"" start=""12/09/2018 1..."
3,3485219,80554583,valid,"<record><visit type=""NULL"" start=""10/18/19 12:..."
4,3485227,80610591,valid,"<record><visit type=""Inpatient Visit"" start=""0..."
...,...,...,...,...
8836,3668561,97171425,valid,"<record><visit type=""NULL"" start=""09/12/03 12:..."
8837,3668598,97172294,valid,"<record><visit type=""NULL"" start=""12/16/12 12:..."
8838,3668601,97172318,valid,"<record><visit type=""Inpatient Visit"" start=""0..."
8839,3668606,97172468,valid,"<record><visit type=""NULL"" start=""12/14/2011 1..."


In [6]:
k = 10000
seed = 1234
reservior_sampler = ReservoirSampler(k, seed)

def sample_from_lumia(path, train_file, path_to_save, i):

    exlude_shard = ['train.0.1684831482.jsonl.gz', 'train.1.1684831482.jsonl.gz']

    if train_file not in exlude_shard:
    
        df = pd.read_json(os.path.join(path, train_file), lines=True, compression='gzip')

        for text in tqdm(df['text']):
            reservior_sampler.add(text)

        results = reservior_sampler.values
        
        with open(path_to_save + f'lumia_pretraining_data_{i}.pkl', 'wb') as f:
            pickle.dump(results, f)

In [7]:
path = '/local-scratch/nigam/projects/zphuo/data/lumia/download_frazier/'
path_to_save = '/local-scratch/nigam/projects/zphuo/data/medical_instruction/'

tasks = [(path, train_file, path_to_save, i) for i, train_file in enumerate(os.listdir(path)) if 'train' in train_file]

ctx = multiprocessing.get_context("forkserver")

num_threads = len([train_file for train_file in os.listdir(path) if 'train' in train_file])
with ctx.Pool(num_threads) as pool:
    parallel_result = list(pool.imap(sample_from_lumia, tasks))

Process ForkServerPoolWorker-2:
Process ForkServerPoolWorker-1:
Process ForkServerPoolWorker-9:
Process ForkServerPoolWorker-3:
Process ForkServerPoolWorker-20:
Process ForkServerPoolWorker-4:
Process ForkServerPoolWorker-10:
Traceback (most recent call last):
Process ForkServerPoolWorker-5:
  File "/home/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'sample_from_lumia' on <module '__main__' (built-in)>
Process ForkServerPoolWorker-6:
Traceback (most recent call last):
  Fi

In [4]:
path_to_save = '/local-scratch/nigam/projects/zphuo/data/medical_instruction/'
import pickle
with open(path_to_save + 'lumia_pretraining_data_139766.pkl', 'rb') as f:
    results = pickle.load(f)

In [6]:
results[0]

'<record><visit type="Emergency Room Visit" start="09/06/2012 00:00"><day start="09/06/2012 00:00"><person>Birth: 04/26/2007\nGender: MALE\nRace: White\nEthnicity: Hispanic or Latino\nAge in Days: 1960\nAge in Years: 5</person><measurement start="09/06/2012 19:28"><code>[LOINC/8462-4] Diastolic blood pressure 86</code><code>[LOINC/38214-3] Pain severity [Score] Visual analog score 10</code><code>[LOINC/8480-6] Systolic blood pressure 127</code><code>[LOINC/8478-0] Mean blood pressure 100 mmHg</code><code>[LOINC/9279-1] Respiratory rate 20</code><code>[LOINC/29463-7] Body weight 578.49</code><code>[LOINC/8310-5] Body temperature 98.20</code><code>[LOINC/8867-4] Heart rate 113</code></measurement><observation start="09/06/2012 19:28"><code>[LOINC/8327-9] Body temperature measurement site 1</code><code>[LOINC/LP21258-6] Oxygen saturation 100 %</code></observation><condition_occurrence start="09/06/2012 20:27"><code>[ICD/789.00] Abdominal pain</code></condition_occurrence><visit_detail sta